In [148]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import janitor
import math
import time
import warnings
import datetime
import matplotlib.pyplot as plt

In [149]:
warnings.filterwarnings("ignore")

In [150]:
def find_tables_flex(first, last, team, year):
    for i in range(0,8):
        try:
            if len(last) < 4:
                last = last + ('x' * (4 - len(last)))
            url = 'https://www.pro-football-reference.com/players/%s/%s%s0%i.htm' % (last[0], last[0:4], 
                                                                                     first[0:2], i)
            if first == 'David' and last == 'Johnson':
                url = 'https://www.pro-football-reference.com/players/J/JohnDa08.htm'
            response = requests.get(url)
            x = pd.read_html(url)
            df = x[0]
            df[df.columns[0]] = df[df.columns[0]].str.replace(r'[\*+]', '', regex=True)
            if (df[df[df.columns[0]] == str(year)][df.columns[2]]).values == team:
                return x[0], response, i + 1
        except:
            pass
    return 0, 0, i + 1

def find_age(response, year):
    soup = BeautifulSoup(response.content, "html.parser")
    x = soup.find('div', {'id': 'info'})
    dob = x.find('span"', {'id': 'necro-birth'})['data-birth']
    return math.floor(((datetime.date(year, 9, 7) - datetime.date(int(dob[0:4]), int(dob[5:7]), int(dob[8:10]))).days)/365)

In [151]:
rbs = pd.read_csv('rbs.csv')

In [152]:
rbs.drop(columns = 'Unnamed: 0', inplace = True)

In [153]:
rb2022 = pd.read_csv('2022rbs.csv')
corr_teams = pd.read_html('https://www.pro-football-reference.com/years/2022/fantasy.htm')
corrrb2022 = corr_teams[0][[('Unnamed: 1_level_0',  'Player'), ('Unnamed: 2_level_0',      'Tm'), ('Unnamed: 3_level_0', 'FantPos')]]

In [154]:
corrrb2022[('Unnamed: 1_level_0',  'Player')] = corrrb2022[('Unnamed: 1_level_0',  'Player')].str.replace(r'[\*+]', '', 
                                                                                                regex=True)

In [155]:
corrrb2022 = corrrb2022[corrrb2022[('Unnamed: 3_level_0', 'FantPos')] == 'RB']

In [156]:
rb2022['Player'] = rb2022['Player'].replace('III', '', regex = True)
rb2022['Player'] = rb2022['Player'].replace('II', '', regex = True)
rb2022['Player'] = rb2022['Player'].replace('Jr.', '', regex = True)

In [157]:
x = rb2022['Player'].str.split(' ', expand = True)

In [158]:
for i in range(0, len(x)):
    if x.loc[i,2] == "":
        x.loc[i,2] = x.loc[i,3]
x.drop(columns = 3, inplace = True)
x[2] = x[2].replace(r'[()]', '', regex = True)

In [159]:
rb2022[['First', 'Last', 'Tm']] = x
rb2022['Player'] = [i.strip() if isinstance(i,str) else i for i in rb2022['Player'].str.split('(', expand = True)[0]]
rb2022['Year'] = 2022
rb2022['Pos'] = 'RB'
rb2022.dropna(inplace = True)
rb2022['Age'] = None
rb2022.drop(columns = 'Rank', inplace = True)

In [160]:
x = pd.DataFrame()
x['Player'] = corrrb2022[('Unnamed: 1_level_0',  'Player')]
x['Tm'] = corrrb2022[('Unnamed: 2_level_0',      'Tm')]
x['Pos'] = corrrb2022[('Unnamed: 3_level_0', 'FantPos')]

In [161]:
rb2022 = pd.merge(rb2022, x, on = 'Player')

In [162]:
rb2022.drop(columns = ['Tm_x', 'Pos_x'], inplace = True)
rb2022.rename(columns = {'Tm_y': 'Tm', 'Pos_y': 'Pos'}, inplace = True)

In [163]:
column_order = ['First', 'Last', 'Year', 'Tm', 'Pos', 'Age', 'FPTS', 'Player']
rb2022 = rb2022.reindex(columns = column_order)
rb2022.rename(columns = {'FPTS': 'FantasyPoints'}, inplace = True)

In [164]:
totalrbs = pd.concat([rbs, rb2022])
totalrbs = totalrbs.sort_values('FantasyPoints', ascending = False).groupby('Year').head(75)
totalrbs = totalrbs.reset_index().drop(columns = 'index')

In [165]:
totalrbs['First'] = totalrbs['First'].replace('J.K.', 'JK', regex = True)
totalrbs['First'] = totalrbs['First'].replace('Beanie', 'Chris', regex = True)
totalrbs['First'] = totalrbs['First'].replace("\'", '', regex = True)
totalrbs['Last'] = totalrbs['Last'].replace("Jones-Drew", 'Drew', regex = True)

In [167]:
totalrbs['Targets'] = np.nan
totalrbs['Receptions'] = np.nan
totalrbs['RecYards'] = np.nan
totalrbs['RecTD'] = np.nan
totalrbs['Ctch%'] = np.nan
totalrbs['RushAtt'] = np.nan
totalrbs['RushYards'] = np.nan
totalrbs['RushTD'] = np.nan
totalrbs['Fumbles'] = np.nan
totalrbs['Experience'] = np.nan

In [168]:
start = time.time()
for i in range(len(totalrbs)):
    if i % 400 == 0:
        print(i, totalrbs.loc[i, 'First'],totalrbs.loc[i, 'Last'])
    try:    
        stats, response, j = find_tables_flex(totalrbs.loc[i, 'First'],totalrbs.loc[i, 'Last'], 
                                       totalrbs.loc[i, 'Tm'], totalrbs.loc[i, 'Year'])
        stats[(( 'Unnamed: 0_level_0',  'Year'))] = pd.to_numeric(stats[(( 'Unnamed: 0_level_0',  'Year'))], errors = 'coerce')
        
        if totalrbs.loc[i, 'Tm'] == '2TM':
            idx = stats[(stats[( 'Unnamed: 2_level_0', 'Tm')] == '2TM') & (stats[( 'Unnamed: 0_level_0', 'Year')] == totalrbs.loc[i, 'Year'])].index + 1
            totalrbs.loc[i, 'Tm'] = stats.loc[idx, ( 'Unnamed: 2_level_0',    'Tm')].iloc[0]
        stats.dropna(subset = [(( 'Unnamed: 1_level_0',   'Age'))], inplace = True)
        
        totalrbs.loc[i, 'Experience'] = len(stats[stats[(( 'Unnamed: 0_level_0',  'Year'))] <= totalrbs.loc[i, 'Year']])
        
        if totalrbs.loc[i, 'Year'] == 2022:
            totalrbs.loc[i, 'Age'] = find_age(response, totalrbs.loc[i, 'Year'])

        date = stats[(stats[(( 'Unnamed: 0_level_0',  'Year'))] < (totalrbs.loc[i, 'Year'])) & (stats[(( 'Unnamed: 0_level_0',  'Year'))] >= (totalrbs.loc[i, 'Year'] - 2))].reset_index()
        date[('Receiving', 'Ctch%')] = date[('Receiving', 'Ctch%')].replace('%', '', regex = True)
        date.drop(columns = [( 'Unnamed: 2_level_0',     'Tm'), 
                             ( 'Unnamed: 3_level_0',     'Pos')], inplace = True)
        if ('Unnamed: 32_level_0', 'Awards') in date.columns:
            date.drop(columns = [('Unnamed: 32_level_0', 'Awards')], inplace = True)
        date = date.astype(float)
        
        totalrbs.loc[i, 'Targets'] = (date[('Receiving', 'Tgt')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalrbs.loc[i, 'Receptions'] = (date[('Receiving', 'Rec')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalrbs.loc[i, 'RecYards'] = (date[('Receiving', 'Yds')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalrbs.loc[i, 'RecTD'] = (date[('Receiving', 'TD')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalrbs.loc[i, 'Ctch%'] = (date[('Receiving', 'Ctch%')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalrbs.loc[i, 'RushAtt'] = (date[('Rushing', 'Att')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalrbs.loc[i, 'RushYards'] = (date[('Rushing', 'Yds')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalrbs.loc[i, 'RushTD'] = (date[('Rushing', 'TD')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games','G')]*(date.index+1)).sum()
        totalrbs.loc[i, 'Fumbles'] = (date[('Unnamed: 30_level_0', 'Fmb')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
    except:
        print("Error:", totalrbs.loc[i, 'First'], totalrbs.loc[i, 'Last'], totalrbs.loc[i, 'Tm'], totalrbs.loc[i, 'Year'])
    time.sleep(7.5 * j)
print(time.time() - start)

0 LaDainian Tomlinson
Error: Javonte Williams DEN 2021
Error: Domanick Williams HOU 2005
Error: Darrel Williams KAN 2021
400 Lamar Miller
Error: Michael Carter NYJ 2021
Error: Michael Carter NYJ 2022
Error: Josh Cribbs CLE 2011
Error: Breece Hall NYJ 2022
800 Wayne Gallman
Error: Dwayne Harris NYG 2015
Error: Jabar Gaffney NWE 2008
Error: Isaac Redman PIT 2012
Error: Isaac Redman PIT 2011
Error: Ben Tate 2TM 2014
1200 Darren Sproles
Error: Darrel Williams KAN 2020
Error: Isaac Redman PIT 2010
16293.89394569397


In [170]:
totalrbs.dropna(inplace = True)

In [171]:
totalrbs.to_csv('finalrbs.csv')

In [172]:
totalrbs

,First,Last,Year,Tm,Pos,Age,FantasyPoints,Player,Targets,Receptions,RecYards,RecTD,Ctch%,RushAtt,RushYards,RushTD,Fumbles,Experience
0,LaDainian,Tomlinson,2006,SDG,RB,27.0,481.1,LaDainian Tomlinson,73.489362,51.638298,392.659574,1.680851,70.700000,339.000000,1421.468085,17.680851,3.957447,6.0
1,Christian,McCaffrey,2019,CAR,RB,23.0,469.2,Christian McCaffrey,120.333333,98.000000,795.000000,5.666667,81.133333,185.000000,877.000000,5.333333,3.333333,3.0
2,Steven,Jackson,2006,STL,RB,23.0,415.4,Steven Jackson,45.136364,35.363636,278.318182,1.363636,78.518182,215.818182,927.318182,6.727273,2.363636,3.0
3,David,Johnson,2016,ARI,RB,25.0,405.8,David Johnson,57.000000,36.000000,457.000000,4.000000,63.200000,125.000000,581.000000,8.000000,4.000000,2.0
4,Arian,Foster,2010,HOU,RB,24.0,392.0,Arian Foster,9.000000,8.000000,93.000000,0.000000,88.900000,54.000000,257.000000,3.000000,1.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,Brad,Hoover,2006,CAR,RB,30.0,41.5,Brad Hoover,24.500000,16.227273,110.545455,0.636364,66.350000,28.454545,93.272727,0.000000,0.000000,7.0
1346,Marcel,Shipp,2006,ARI,RB,28.0,40.1,Marcel Shipp,44.000000,35.000000,255.000000,0.000000,79.500000,157.000000,451.000000,0.000000,4.000000,5.0
1347,Aaron,Stecker,2006,NOR,RB,31.0,39.1,Aaron Stecker,44.521739,32.913043,243.782609,0.000000,74.082609,82.130435,321.608696,0.695652,2.304348,7.0
1348,Mike,Anderson,2006,BAL,RB,33.0,38.7,Mike Anderson,21.000000,18.000000,212.000000,1.000000,85.700000,239.000000,1014.000000,12.000000,2.000000,6.0


In [113]:
totalrbs2 = totalrbs.sort_values('FantasyPoints', ascending = False).groupby('Year').head(50)

In [144]:
totalrbs = pd.merge(totalrbs, rbs[['First', 'Last', 'Year', 'Tm', 'FantasyPoints']], on = ['First', 'Last', 'Year', 'Tm'], how = 'left')